In [3]:
#导入数据集iris  
from sklearn.datasets import load_iris   
  
#载入数据集  
iris = load_iris()  
#特征矩阵
print(iris.data[0:5])  
#目标向量
# print(iris.target)
from sklearn.preprocessing import StandardScaler
#标准化，返回值为标准化后的数据
X_scalerd = StandardScaler().fit_transform(iris.data)
# print(X_scalerd[0:5])
# print(X_scalerd.mean(axis=0))
# print(X_scalerd.std(axis=0))
from sklearn.preprocessing import MinMaxScaler
#区间缩放，返回值为缩放到[0, 1]区间的数据
# print(MinMaxScaler().fit_transform(iris.data)[0:5])

from sklearn.preprocessing import Normalizer
#归一化，返回值为归一化后的数据
# print(Normalizer().fit_transform(iris.data)[0:5])

from sklearn.feature_selection import VarianceThreshold
#方差选择法，返回值为特征选择后的数据
#参数threshold为方差的阈值
print(VarianceThreshold(threshold=3).fit_transform(iris.data)[0:5])


from sklearn.feature_selection import SelectKBest
from scipy.stats import pearsonr
from numpy import vstack, array, nan
#选择K个最好的特征，返回选择特征后的数据
#第一个参数为计算评估特征是否好的函数，该函数输入特征矩阵和目标向量，输出二元组（评分，P值）的数组，数组第i项为第i个特征的评分和P值。在此定义为计算相关系数
#参数k为选择的特征个数
# SelectKBest(lambda X, Y: array(map(lambda x:pearsonr(x, Y), X.T)).T, k=2).fit_transform(iris.data,iris.target)
print(SelectKBest(lambda X, Y: array(map(lambda x:pearsonr(x, Y), X.T)).T, k=2))

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#选择K个最好的特征，返回选择特征后的数据
print(SelectKBest(chi2, k=2).fit_transform(iris.data, iris.target)[0:5])


from sklearn.feature_selection import SelectKBest
from minepy import MINE
 
#由于MINE的设计不是函数式的，定义mic方法将其为函数式的，返回一个二元组，二元组的第2项设置成固定的P值0.5
def mic(x, y):
 m = MINE()
 m.compute_score(x, y)
 return (m.mic(), 0.5)

#选择K个最好的特征，返回特征选择后的数据
a = SelectKBest(lambda X, Y: array(map(lambda x:mic(x, Y), X.T)).T, k=2)
a.fit_transform(iris.data, iris.target)

[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]]
[[ 1.4]
 [ 1.4]
 [ 1.3]
 [ 1.5]
 [ 1.4]]
SelectKBest(k=2, score_func=<function <lambda> at 0x000000001B3946D8>)
[[ 1.4  0.2]
 [ 1.4  0.2]
 [ 1.3  0.2]
 [ 1.5  0.2]
 [ 1.4  0.2]]


IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

iris = load_iris()

X, y = iris.data, iris.target

# This dataset is way too high-dimensional. Better do PCA:
pca = PCA(n_components=2)

# Maybe some original features where good, too?
selection = SelectKBest(k=1)

# Build estimator from PCA and Univariate selection:

combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])

# Use combined features to transform dataset:
X_features = combined_features.fit(X, y).transform(X)

svm = SVC(kernel="linear")

# Do grid search over k, n_components and C:

pipeline = Pipeline([("features", combined_features), ("svm", svm)])

param_grid = dict(features__pca__n_components=[1, 2, 3],
                  features__univ_select__k=[1, 2],
                  svm__C=[0.1, 1, 10])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=10)
grid_search.fit(X, y)
print(grid_search.best_estimator_)